pymorphy - для токенизации

nltk - для нормализации

fuzzywuzzy - для нечеткого поиска (для законов)

- pymorphy3 - морфологический анализ русского языка
- NLTK - токенизация и обработка текста
- fuzzywuzzy - нечеткий поиск для сопоставления названий законов
- Pydantic - валидация и структурирование данных
- re - регулярные выражения для парсинга

In [22]:
!pip install pymorphy3
!pip install fuzzywuzzy
!pip install pydantic

In [23]:
import pymorphy3 as pym
import re

In [24]:
from fuzzywuzzy import fuzz #нечеткий поиск
from fuzzywuzzy import process

In [25]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
stopwords_ru = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /Users/a1111/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/a1111/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
text = "В ходе проверки декларации о доходах государственного служащего выявлены несоответствия, которые, согласно подп. 12 п. 23 ст. 164 Указа Президента РФ от 02.04.2013 «О мерах по реализации отдельных положений Федерального закона \"О контроле за соответствием расходов лиц, замещающих государственные должности, и иных лиц их доходам\"», подлежат дополнительному анализу. В связи с этим, было принято решение провести расширенное расследование в отношении указанных расходов."

In [27]:
from pymorphy3 import MorphAnalyzer

def normalize_text(text: str) -> str:
    morph = MorphAnalyzer()
    tokens = word_tokenize(text.lower(), language='russian')

    normalized_tokens = []
    for token in tokens:
        if re.match(r'^\d+\.\d+', token) or re.match(r'^\d+(?:\.\d+)+$', token):
            normalized_tokens.append(token)
        elif token.isdigit() or (any(c.isdigit() for c in token) and any(c.isalpha() for c in token)):
            normalized_tokens.append(token)
        elif token in ['ст', 'п', 'пп', 'ст.', 'п.', 'пп.', 'нк', 'гк', 'ук', 'тк', 'апк', 'бк', 'коап', 'рф', 'ч']:
            normalized_tokens.append(token.split('.')[0])
        elif token.isalpha() and token not in stopwords_ru:
            parsed = morph.parse(token)[0]
            normalized_tokens.append(parsed.normal_form)
        # удаляем точки, которые не являются частью чисел (знаки препинания)
        elif token != '.':
            normalized_tokens.append(token)

    return ' '.join(normalized_tokens)

In [28]:
import json

with open('law_aliases.json', 'r', encoding='utf-8') as f:
    law_aliases = json.load(f)

In [29]:
law_aliases_invers = {i:k for k,v in law_aliases.items() for i in v}

In [30]:
from pydantic import BaseModel
import json
from typing import List, Optional, Dict

class LawLink(BaseModel):
    law_id: Optional[int] = None
    article: Optional[str] = None
    point_article: Optional[str] = None
    subpoint_article: Optional[str] = None

In [31]:
def extract_multiple_entities(raw: str):
    clean_subpoint = re.sub(r'[^\dа-я,\s\.\-]', '', raw.lower())

    clean_subpoint = re.sub(r'(?<!\d)\.(?!\d)', '', clean_subpoint)

    parts = re.split(r'[,и]', clean_subpoint)

    entities = []
    for part in parts:
        part = part.strip()
        if part:
            entities.append(part)

    return entities if entities else [raw]

In [32]:
def find_law_id_fuzzy(law_name):
    max_score = 0
    for law_key in law_aliases_invers.keys():
        lev_score = fuzz.partial_ratio(law_name, law_key.lower())
        if lev_score > max_score:
            max_score = lev_score
            res_key = law_key
    if max_score < 90:
        return None
    else:
        return law_aliases_invers[res_key]


def process_match(match, context):
    references = []
    groups = match.groups()

    articles = []
    point_articles = []
    subpoint_articles = []

    if match.group('пункт_номера') is  None:
      point_articles = [None]
      if match.group('часть_номера') is not None:
        if ',' in match.group('часть_номера') or 'и' in match.group('часть_номера'):
          point_articles = extract_multiple_entities(match.group('часть_номера'))
        else:
          point_articles = [match.group('часть_номера')]
      else:
        point_articles = [None]
    else:
      if ',' in match.group('пункт_номера') or 'и' in match.group('пункт_номера'):
          point_articles = extract_multiple_entities(match.group('пункт_номера'))
      else:
          point_articles = [match.group('пункт_номера')]

    if match.group('подпункт_номера') is  None:
      subpoint_articles = [None]
    else:
      if ',' in match.group('подпункт_номера') or 'и' in match.group('подпункт_номера'):
          subpoint_articles = extract_multiple_entities(match.group('подпункт_номера'))
      else:
          subpoint_articles = [match.group('подпункт_номера')]

    if match.group('статья_номера') is  None:
      articles = [None]
    else:
      if ',' in match.group('статья_номера') or 'и' in match.group('статья_номера'):
          articles = extract_multiple_entities(match.group('статья_номера'))
      else:
          articles = [match.group('статья_номера')]

    law_id = find_law_id_fuzzy(match.group('остальное'))


    for article in articles:
      for point_article in point_articles:
        for subpoint_article in subpoint_articles:
          reference = LawLink(
                      law_id=law_id,
                      article=article,
                      point_article=point_article,
                      subpoint_article=subpoint_article)
          print(reference)

          references.append(reference)


    return references

In [33]:
def find_references_in_text(original_text: str) -> List[LawLink]:
    references = []

    patterns = [
    # Паттерн для парсинга структуры ссылки
    r'(?:в\s+)?(?:(?P<подпункт_ключ>пп\.|подпункт[а-я]{0,7}|подп\.)\s*(?P<подпункт_номера>(?:\d{1,4}[а-я]?|[а-я])(?:\s*,\s*(?:\d{1,4}[а-я]?|[а-я]))*(?:\s*и\s*(?:\d{1,4}[а-я]?|[а-я]))?)\s+)?'
    r'(?:в\s+)?(?:(?P<пункт_ключ>п\.|пункт[а-я]{0,5}|пунт[а-я]{0,5})\s*(?P<пункт_номера>(?:\d{1,4}(?:[\.\-]\d{1,3})*[а-я]?|[а-я])(?:\s*,\s*(?:\d{1,4}(?:[\.\-]\d{1,3})*[а-я]?|[а-я]))*(?:\s*и\s*(?:\d{1,4}(?:[\.\-]\d{1,3})*[а-я]?|[а-я]))?)\s+)?'
    r'(?:в\s+)?(?:(?P<часть_ключ>ч\.|част[ьи])\s*(?P<часть_номера>(?:\d{1,3}(?:\.\d{1,3})?|[а-я])(?:\s*,\s*(?:\d{1,3}(?:\.\d{1,3})?|[а-я]))*(?:\s*и\s*(?:\d{1,3}(?:\.\d{1,3})?|[а-я]))?)(?:\s*,\s*)?\s+)?'
    r'(?:в\s+)?(?:(?P<статья_ключ>ст\.|стать[ейиюя]|статей?|статья)\s*(?:(?:в|на|по)\s+)?(?P<статья_номера>(?:\d{1,4}(?:[\.\-]\d{1,3})*[а-я]?)(?:\s*,\s*(?:\d{1,4}(?:[\.\-]\d{1,3})*[а-я]?))*(?:\s*и\s*(?:\d{1,4}(?:[\.\-]\d{1,3})*[а-я]?))?)\s+)?'
    r'(?P<остальное>(?:'
    # Кодексы
    r'(?:Арбитражн(?:ого|ый)\s+процессуальн(?:ого|ый)|Бюджетн(?:ого|ый)|Водн(?:ого|ый)|Воздушн(?:ого|ый)|Градостроительн(?:ого|ый)|Гражданск(?:ого|ий)|Гражданск(?:ого|ий)\s+процессуальн(?:ого|ый)|Жилищн(?:ого|ый)|Семейн(?:ого|ый)|Таможенн(?:ого|ый)|Трудов(?:ого|ой)|Уголовно-исполнительн(?:ого|ый)|Уголовно-процессуальн(?:ого|ый)|Уголовн(?:ого|ый)|Лесн(?:ого|ой)|Налогов(?:ого|ый)|Земельн(?:ого|ый))\s+кодекс(?:а|)(?:\s+Российской Федерации|\s+России|\s+РФ|)'
    # Аббревиатуры
    r'|АПК(?:\s+(?:России|РФ))?|БК(?:\s+(?:России|РФ))?|ГК(?:\s+РФ)?|ГПК(?:\s+(?:России|РФ))?|ЖК(?:\s+(?:России|РФ))?|СК(?:\s+(?:России|РФ))?|ТК(?:\s+РФ)?'
    r'|УИК(?:\s+(?:России|РФ))?|УПК(?:\s+(?:России|РФ))?|УК(?:\s+(?:России|РФ))?|ЛК(?:\s+(?:России|РФ))?|НК(?:\s+(?:России|РФ))?|ЗК(?:\s+(?:России|РФ))?'
    # Кодексы об административных правонарушениях
    r'|Кодекс(?:а|)(?:\s+Российской Федерации|\s+России|\s+РФ|)?\s+об\s+административных\s+правонарушениях|КоАП(?:\s+Российской Федерации|\s+России|\s+РФ|)?'
    # Другие кодексы
    r'|Кодекс(?:а|)\s+административного\s+судопроизводства(?:\s+Российской Федерации|\s+России|\s+РФ|)?'
    r'|Кодекс(?:а|)\s+внутреннего\s+водного\s+транспорта(?:\s+Российской Федерации|\s+России|\s+РФ|)?'
    r'|Кодекс(?:а|)\s+торгового\s+мореплавания(?:\s+Российской Федерации|\s+России|\s+РФ|)?'
    # Указы Президента
    r'|Указ(?:а|)(?:\s+Президента(?:\s+Российской Федерации|\s+России|\s+РФ|)?)?(?:\s+(?:№?\s*\d+|\s*от\s*\d{2}\.\d{2}\.\d{4}))?(?:\s*«[^»]*»)?'
    # Распоряжения Президента
    r'|Распоряжени(?:я|е)(?:\s+Президента(?:\s+Российской Федерации|\s+России|\s+РФ|)?)?(?:\s+(?:№?\s*\d+-\s*рп|от\s*\d{2}\.\d{2}\.\d{4}))?(?:\s*«[^»]*»)?'
    r'|РП(?:\s+(?:№?\s*\d+-\s*рп|от\s*\d{2}\.\d{2}\.\d{4}))?(?:\s*«[^»]*»)?'
    # Федеральные законы
    r'|Федеральн(?:ого|ый)\s+закон(?:а|)(?:\s+[^,\.;]*)?'
    r'|ФЗ(?:\s+[^,\.;]*)?'

    r'|Закона\s+«[^»]*»'
    r'|Закона\s+"[^"]*"'
    # Конституция
    r'|Конституци(?:и|я)(?:\s+РФ|\s+России|\s+Российской Федерации)?'
    # Основы законодательства
    r'|Основы\s+законодательства(?:\s+(?:Российской\s+Федерации|России|РФ))?(?:\s+№?\s*\d+(?:-I)?)?(?:\s+от\s+\d{2}\.\d{2}\.\d{4})?(?:\s*«[^»]*»)?'
    # Законы Российской Федерации
    r'|Закон(?:\s+(?:Российской\s+Федерации|России|РФ))?(?:\s+№?\s*\d+(?:-I)?)?(?:\s+от\s+\d{2}\.\d{2}\.\d{4})?(?:\s*«[^»]*»)?'
    # Федеральные стандарты бухгалтерского учета
    r'|Федеральный\s+стандарт\s+бухгалтерского\s+учета(?:\s+(?:государственных\s+финансов|для\s+организаций\s+государственного\s+сектора))?(?:\s+ФСБУ\s*\d+(?:\/\d{4})?)?(?:\s*«[^»]*»)?'
    r'|ФСБУ(?:\s+(?:государственных\s+финансов|для\s+организаций\s+государственного\s+сектора))?(?:\s*\d+(?:\/\d{4})?)?(?:\s*«[^»]*»)?'
    # Положения по бухгалтерскому учету
    r'|Положение\s+по\s+бухгалтерскому\s+учету(?:\s+ПБУ\s*\d+(?:\/\d{4})?)?(?:\s*«[^»]*»)?'
    r'|ПБУ(?:\s*\d+(?:\/\d{4})?)?(?:\s*«[^»]*»)?'
    # Положение по ведению бухгалтерского учета
    r'|Положени(?:я|е)\s+по\s+ведению\s+бухгалтерского\s+учета\s+и\s+бухгалтерской\s+отчетности\s+в\s+Российской\s+Федерации'

    r')'
    r'(?=\s|,|\.|;|$))'
]


    all_matches = []
    for pattern in patterns:
        matches = re.finditer(pattern, original_text, re.IGNORECASE)
        all_matches.append(matches)

    # Удаляем дубликаты и выводим
    unique_matches = list(dict.fromkeys(all_matches))

    iter = 0
    for matches in unique_matches:
      for match in matches:
        iter += 1
        print(iter, match.groups())
        references.extend(process_match(match, original_text))

    return references

In [34]:
def extract_legal_references_advanced(text: str):
    references = []

    #norm_text = normalize_text(text)

    sentence_references = find_references_in_text(text)
    references.extend(sentence_references)

    return references

In [46]:
text_2="Во время проверки воздушного судна была обнаружена необходимость уточнения некоторых процедур. В соответствии с пп. и п. 24 ст. 489 Воздушного кодекса РФ, инспектор обязал авиакомпанию провести дополнительные меры безопасности. Эти меры направлены на обеспечение максимальной защиты пассажиров и экипажа."

In [47]:
# рандомного кодекса не учитывается, так как это ложная ссылка
result = extract_legal_references_advanced(text_2)

1 ('пп.', 'и', 'п.', '24', None, None, 'ст.', '489', 'Воздушного кодекса РФ')
law_id=None article='489' point_article='24' subpoint_article='и'


In [35]:
# рандомного кодекса не учитывается, так как это ложная ссылка
result = extract_legal_references_advanced(text)

1 ('подп.', '12', 'п.', '23', None, None, 'ст.', '164', 'Указа Президента РФ от 02.04.2013 «О мерах по реализации отдельных положений Федерального закона "О контроле за соответствием расходов лиц, замещающих государственные должности, и иных лиц их доходам"»')
law_id=218 article='164' point_article='23' subpoint_article='12'


In [36]:
result

[LawLink(law_id=218, article='164', point_article='23', subpoint_article='12')]

In [37]:
norm_text = normalize_text(text)
norm_text

"в ход проверка декларация о доход государственный служащий выявить несоответствие , который , согласно подп. 12 п 23 ст 164 указ президент рф от 02.04.2013 « о мера по реализация отдельный положение федеральный закон `` о контроль за соответствие расход лицо , замещать государственный должность , и иной лицо их доход '' » , подлежать дополнительный анализ в связь с это , было принять решение провести расширить расследование в отношение указанный расход"

In [38]:
answers = ['пп. 1 п. 1 ст. 374 НК РФ',
'ст. 105 УК РФ',
'п.  10 АПК',
'статье 17 Конституции Российской Федерации',
'п. 5 ст. 105 УК РФ',
'п. 3 ст. 158 УК РФ',
'ст. 30 Гражданского кодекса Российской Федерации',
'пп. 12 п. 4 ст. 159 УК РФ',
'п. 2 ст. 228 УК РФ',
'ст. 275 Налогового Кодекса Российской Федерации',
' ст. 90 Семейного кодекса Российской Федерации',
'ст. 70 Трудового кодекса Российской Федерации',
'с п. 1 ст. 213 Гражданского процессуального кодекса',
'статье 16 Закона "О защите прав потребителей"',
'п. 2 ст. 14 Закона "О персональных данных"',
'ст. 19 Федерального закона "О бухгалтерском учёте",',
'п. р ст. 9 Федерального закона "О государственной регистрации юридических лиц и индивидуальных предпринимателей"',
'п. 1 ст. 16 Закона "О защите прав юридических лиц и индивидуальных предпринимателей при осуществлении государственного контроля (надзора) и муниципального контроля"',
'2 пунта б статьи 22 Федерального закона "О государственной гражданской службе Российской Федерации"',
'ст. 12 Федерального закона "О защите конкуренции"',
'Согласно п. 3 статьи 20 Федерального закона "О полиции"',
'пп. 4, 5, 6 и 8 п. 1 ст. 14 Федерального закона "О государственной регистрации недвижимости"',
'ст. 37 ФЗ №20-456',
'ст. 6 Федерального закона "О банках и банковской деятельности"',
'пункте 1 статьи 34 Гражданского Кодекса',
'пп. 1 п. 2 ст. 34 рандомного кодекса ',
'подпунктах а, б и с пункта 3.345, 23 в статье 66 НК РФ',
'3, ст. 30.1 КоАП РФ',
'ст. 211 АПК РФ ']

In [39]:
len(answers)

29

In [40]:
len(result)

1

In [41]:
result

[LawLink(law_id=218, article='164', point_article='23', subpoint_article='12')]